<a href="https://colab.research.google.com/github/sergekamanzi/Chat-Bot-/blob/main/chatbot2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Dataset as a list of dictionaries
rwanda_history_dataset = [
    {
        "Question": "When is human occupation of Rwanda thought to have begun?",
        "Answer": "Human occupation of Rwanda is thought to have begun shortly after the last ice age, around ten thousand years ago in the Neolithic period."
    },
    {
        "Question": "Who were the earliest inhabitants of Rwanda?",
        "Answer": "The earliest inhabitants of Rwanda were the Twa, a group of Pygmy forest hunters and gatherers."
    },
    {
        "Question": "What evidence supports early settlement in Rwanda?",
        "Answer": "Archaeological excavations from the 1950s revealed sparse settlement by hunter-gatherers in the late Stone Age, followed by a larger population of early Iron Age settlers who manufactured dimpled pottery, iron tools, and implements."
    },
    {
        "Question": "Which groups settled in northern and western Rwanda by 3,000 BC?",
        "Answer": "Central Sudanic-speaking farmers and herders settled in northern and western Rwanda by 3,000 BC."
    },
    {
        "Question": "When did the Mashariki Bantu arrive in Rwanda?",
        "Answer": "The Mashariki Bantu arrived on the western borderlines of Rwanda in the Kivu rift valley around 1,000 BC."
    },
    {
        "Question": "What happened to the Nilo-Saharan and South Cushitic groups in Rwanda?",
        "Answer": "The Great Lakes Bantu peoples assimilated all Nilo-Saharan and South Cushitic groups in Rwanda by 500 BC, making the land entirely Bantu-speaking."
    },
    {
        "Question": "Who began to clear forests for permanent settlements in Rwanda?",
        "Answer": "The Hutu, a Bantu group and ancestors of the agriculturalist ethnic group, began to clear forests for permanent settlements."
    },
    {
        "Question": "By what century had the Bantu-speakers in Rwanda organized into small states?",
        "Answer": "By the 15th century, many Bantu-speakers, including Hutu and Tutsi, had organized into small states."
    },
    {
        "Question": "What were the three small states in Rwanda by the 15th century?",
        "Answer": "The three small states were an unnamed state established by the Renge lineages of the Singa clan, the Mubari state of the Zigaba clan, and the Gisaka state in southeast Rwanda."
    },
    {
        "Question": "Who was Mwami Rwabugiri, and what did he do in the 19th century?",
        "Answer": "Mwami Rwabugiri was a king of the Kingdom of Rwanda who, in the 19th century, conducted military conquests and administrative consolidation, bringing most of modern Rwanda under the kingdom's control."
    },
    {
        "Question": "Which colonial powers allied with the Rwandan court in the 19th century?",
        "Answer": "The colonial powers Germany and Belgium allied with the Rwandan court in the 19th century."
    },
    {
        "Question": "What was the result of the convergence of anti-colonial and anti-Tutsi sentiment in Rwanda?",
        "Answer": "The convergence of anti-colonial and anti-Tutsi sentiment led Belgium to grant Rwanda national independence in 1962."
    },
    {
        "Question": "Who became the first president of Rwanda after independence in 1962?",
        "Answer": "Grégoire Kayibanda became the first elected president of Rwanda after independence in 1962."
    },
    {
        "Question": "What event worsened ethnic and political tensions in Rwanda in 1973?",
        "Answer": "Juvénal Habyarimana, a Hutu, seized power in 1973, worsening ethnic and political tensions."
    },
    {
        "Question": "What group invaded Rwanda in 1990, starting the Rwandan Civil War?",
        "Answer": "The Rwandan Patriotic Front (RPF), a rebel group composed of 10,000 Tutsi refugees, invaded Rwanda in 1990, starting the Rwandan Civil War."
    },
    {
        "Question": "What was the catalyst for the 1994 Rwandan genocide?",
        "Answer": "The assassination of President Juvénal Habyarimana in 1994 was the catalyst for the Rwandan genocide."
    },
    {
        "Question": "How many people were killed during the 1994 Rwandan genocide?",
        "Answer": "Between 500,000 and 1,000,000 Tutsis and moderate Hutus were killed during the 1994 Rwandan genocide."
    },
    {
        "Question": "Who led the RPF during the 1994 genocide and civil war?",
        "Answer": "Paul Kagame led the Rwandan Patriotic Front (RPF) during the 1994 genocide and civil war."
    },
    {
        "Question": "What happened to Hutu refugees after the RPF conquered Rwanda in 1994?",
        "Answer": "Approximately two million Hutu refugees fled to neighboring countries like Burundi, Tanzania, Uganda, and Zaire, fearing Tutsi retribution."
    },
    {
        "Question": "What was the Great Lakes refugee crisis?",
        "Answer": "The Great Lakes refugee crisis refers to the exodus of about two million Hutu refugees to neighboring countries after the RPF's victory in 1994."
    },
    {
        "Question": "What role did Rwanda play in the First Congo War?",
        "Answer": "Rwanda, led by the RPF, invaded Zaire in 1996 to attack Hutu Interahamwe forces, allying with Laurent Kabila to overthrow Mobutu Sese Seko in the First Congo War."
    },
    {
        "Question": "What was the outcome of the Second Congo War for Rwanda?",
        "Answer": "Rwandan forces, along with Ugandan forces, attempted to control parts of the DRC but were pushed back by Angola and Zimbabwe, ending the war in 2002."
    },
    {
        "Question": "What traditional justice system was used in Rwanda before colonial times?",
        "Answer": "The Gacaca justice system was used in Rwanda to resolve conflicts, render justice, and promote reconciliation."
    },
    {
        "Question": "How did the Gacaca system change under colonial rule?",
        "Answer": "Under colonial rule, the Gacaca system became biased against Hutus, with Tutsis often receiving lighter punishments for crimes compared to Hutus."
    },
    {
        "Question": "What was the Kwihutura process in pre-colonial Rwanda?",
        "Answer": "Kwihutura was a process where Hutus who obtained cattle could be considered Tutsi, climbing the social strata, with permission from the Mwami."
    },
    {
        "Question": "How did colonial powers classify Rwandans into ethnic groups?",
        "Answer": "The Belgians classified Rwandans as Hutu, Tutsi, or Twa based on cattle ownership, with those owning ten or more cattle considered Tutsi."
    },
    {
        "Question": "When did Rwanda become a Belgian League of Nations mandate?",
        "Answer": "Rwanda became a Belgian League of Nations mandate in 1916, as part of Ruanda-Urundi."
    },
    {
        "Question": "What crops did the Belgians introduce to Rwanda to improve food production?",
        "Answer": "The Belgians introduced cassava, maize, and the Irish potato to improve food production for subsistence farmers."
    },
    {
        "Question": "What was the Ruzagayura famine, and when did it occur?",
        "Answer": "The Ruzagayura famine occurred in 1943–44, causing one-fifth to one-third of Rwanda's population to die."
    },
    {
        "Question": "What was the impact of Belgian forced labor policies in Rwanda?",
        "Answer": "Belgian forced labor policies, requiring peasants to grow coffee, were unpopular and led to hundreds of thousands of Rwandans migrating to Uganda."
    },
    {
        "Question": "When did Belgium grant Rwanda full independence?",
        "Answer": "Belgium granted Rwanda full independence on July 1, 1962."
    },
    {
        "Question": "What was the Hutu Manifesto, and who wrote it?",
        "Answer": "The Hutu Manifesto was written by Grégoire Kayibanda in 1957, advocating for Hutu emancipation."
    },
    {
        "Question": "What event marked the end of the Tutsi monarchy in Rwanda?",
        "Answer": "The Rwandan Revolution of 1959 and the deposition of King Kigeli V in 1961 marked the end of the Tutsi monarchy."
    },
    {
        "Question": "What was the outcome of the 1961 referendum in Rwanda?",
        "Answer": "The 1961 referendum resulted in Rwanda becoming a republic, with citizens voting overwhelmingly against maintaining the monarchy."
    },
    {
        "Question": "What policies did the Kayibanda government implement to favor Hutus?",
        "Answer": "The Kayibanda government established quotas limiting Tutsi to nine percent of school and civil service positions and continued the use of ethnic identity cards."
    },
    {
        "Question": "When did Juvénal Habyarimana overthrow Grégoire Kayibanda?",
        "Answer": "Juvénal Habyarimana overthrew Grégoire Kayibanda on July 5, 1973."
    },
    {
        "Question": "What political party did Habyarimana form in 1975?",
        "Answer": "Habyarimana formed the National Revolutionary Movement for Development (MRND) in 1975."
    },
    {
        "Question": "What was the Arusha Accords, and when was it signed?",
        "Answer": "The Arusha Accords was a cease-fire agreement signed in August 1993 between the Rwandan government and the RPF to form a power-sharing government."
    },
    {
        "Question": "Who was assassinated alongside Juvénal Habyarimana in 1994?",
        "Answer": "Cyprien Ntaryamira, the Hutu President of Burundi, was assassinated alongside Juvénal Habyarimana in 1994."
    },
    {
        "Question": "What role did Radio Télévision Libre des Mille Collines play in the 1994 genocide?",
        "Answer": "Radio Télévision Libre des Mille Collines broadcast hate speech, inciting violence against Tutsis during the 1994 genocide."
    },
    {
        "Question": "What was Operation Turquoise, and who conducted it?",
        "Answer": "Operation Turquoise was a French military operation in 1994 that occupied parts of Rwanda, intended to prevent mass killings but allegedly allowed Hutu militias to escape."
    },
    {
        "Question": "Who became the de facto leader of Rwanda after the RPF victory in 1994?",
        "Answer": "Paul Kagame became the de facto leader of Rwanda after the RPF victory in 1994, serving as Minister of Defence and Vice-President."
    },
    {
        "Question": "Who was the first Hutu president of Rwanda after the 1994 genocide?",
        "Answer": "Pasteur Bizimungu was the first Hutu president of Rwanda after the 1994 genocide, heading a government of national unity."
    },
    {
        "Question": "What happened to Pasteur Bizimungu in 2000?",
        "Answer": "Pasteur Bizimungu was removed as president in 2000 after becoming critical of the Kagame government, and Paul Kagame took over the presidency."
    },
    {
        "Question": "What is the Vision 2020 development policy in Rwanda?",
        "Answer": "Vision 2020 is a Rwandan government policy aiming to transform the country into a service-based society with a significant middle class by 2020."
    },
    {
        "Question": "What is Rwanda's ranking in the Corruption Perceptions Index as of 2022?",
        "Answer": "As of 2022, Rwanda is the third least corrupt country in Africa, after Botswana and Cape Verde, according to the Corruption Perceptions Index."
    },
    {
        "Question": "What is the Gacaca programme in post-genocide Rwanda?",
        "Answer": "The Gacaca programme is a traditional justice system modified to try lower-level génocidaires, involving community-elected judges and public trials."
    },
    {
        "Question": "When did the Gacaca programme officially conclude in Rwanda?",
        "Answer": "The Gacaca programme officially concluded in June 2012."
    },
    {
        "Question": "What has Rwanda done to address ethnicity post-genocide?",
        "Answer": "Rwanda has outlawed formal discussion of ethnicity to promote healing and unity, with potential trials for those who discuss ethnic groups."
    },
    {
        "Question": "What is Rwanda's role in the President's Emergency Plan for AIDS Relief (PEPFAR)?",
        "Answer": "Rwanda is a PEPFAR focus country, receiving free antiretroviral drugs and food packages for those infected with AIDS."
    },
    {
        "Question": "What economic sectors are key to Rwanda's revenue today?",
        "Answer": "Rwanda earns revenue from coffee and tea exports, with tourism, particularly mountain gorilla visitation, being the main source."
    },
    {
        "Question": "When did Rwanda join the Commonwealth of Nations?",
        "Answer": "Rwanda joined the Commonwealth of Nations in 2009 at the Commonwealth Heads of Government meeting in Port of Spain, Trinidad."
    },
    {
        "Question": "When did Rwanda join the East African Community?",
        "Answer": "Rwanda joined the East African Community in 2009, alongside Burundi."
    },
    {
        "Question": "What human rights concerns have been raised about Rwanda since 2010?",
        "Answer": "Since 2010, concerns include attacks on opposition groups, restrictions on press freedom, and arrests for 'genocide ideology' and 'divisionism.'"
    },
    {
        "Question": "Who was André Kagwa Rwisereka, and what happened to him?",
        "Answer": "André Kagwa Rwisereka, vice president of the Democratic Green Party, was found dead with his head nearly severed in Butare in July 2009."
    },
    {
        "Question": "What happened to journalist Jean-Leonard Rugambage in 2009?",
        "Answer": "Jean-Leonard Rugambage, a journalist for Umuvugizi newspaper, was shot dead outside his home in Kigali in June 2009."
    },
    {
        "Question": "What is the significance of Rwandan coffee in the economy?",
        "Answer": "Rwandan coffee, praised as among the best in the world, has become a significant export, contributing to economic revenue."
    },
    {
        "Question": "What support has Belgium provided to Rwanda's economy?",
        "Answer": "Belgium contributes €25–35 million annually to Rwanda and has supported agriculture and fisheries, including a US$470,000 investment in Lake Kivu fisheries in 2001."
    },
    {
        "Question": "What is the Clinton Hunter Development Initiative in Rwanda?",
        "Answer": "The Clinton Hunter Development Initiative works to improve agricultural productivity, water, sanitation, health services, and international markets in Eastern Rwanda."
    },
    {
        "Question": "What is Rwanda's goal for its economy in the future?",
        "Answer": "Rwanda aims to transform from an agricultural subsistence economy to a knowledge-based economy with high-speed broadband access across the country."
    }
]

In [ ]:
pip install transformers==4.35.0 tensorflow==2.10.0 datasets==2.14.6 nltk==3.8.1 scikit-learn==1.3.2 gradio==4.44.0 numpy==1.24.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.1/123.1 kB 3.3 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement tensorflow==2.10.0 (from versions: 2.12.0rc0, 2.12.0rc1, 2.12.0, 2.12.1, 2.13.0rc0, 2.13.0rc1, 2.13.0rc2, 2.13.0, 2.13.1, 2.14.0rc0, 2.14.0rc1, 2.14.0, 2.14.1, 2.15.0rc0, 2.15.0rc1, 2.15.0, 2.15.0.post1, 2.15.1, 2.16.0rc0, 2.16.1, 2.16.2, 2.17.0rc0, 2.17.0rc1, 2.17.0, 2.17.1, 2.18.0rc0, 2.18.0rc1, 2.18.0rc2, 2.18.0, 2.18.1, 2.19.0rc0, 2.19.0)
ERROR: No matching distribution found for tensorflow==2.10.0


In [3]:
import gradio as gr
import numpy as np
from sentence_transformers import SentenceTransformer, util
import re

# Full Rwanda history text
rwanda_history = """
Human occupation of Rwanda is thought to have begun shortly after the last ice age. By the 11th century,[1] the inhabitants had organized into a number of kingdoms. In the 19th century, Mwami (king) Rwabugiri of the Kingdom of Rwanda conducted a decades-long process of military conquest and administrative consolidation that resulted in the kingdom coming to control most of what is now Rwanda. The colonial powers, Germany and Belgium, allied with the Rwandan court.

A convergence of anti-colonial, and anti-Tutsi sentiment resulted in Belgium granting national independence in 1962. Direct elections resulted in a representative government dominated by the majority Hutu under President Grégoire Kayibanda. Unsettled ethnic and political tensions were worsened when Juvénal Habyarimana, who was also Hutu, seized power in 1973. In 1990, the Rwandan Patriotic Front (RPF), a rebel group composed of 10,000 Tutsi refugees from previous decades of unrest, invaded the country, starting the Rwandan Civil War. The war ground on, worsening ethnic tensions, as the Hutu government feared losing their gains.

The assassination of Habyarimana was the catalyst for the eruption of the 1994 genocide, in which hundreds of thousands of Tutsis and some moderate Hutus were killed, including the prime minister Agathe Uwilingiyimana. The Tutsi RPF conquered Rwanda, and thousands of Hutu were imprisoned pending the establishment of the Gacaca courts.[2] Millions of Hutu fled as refugees, contributing to large refugee camps of Hutu in the neighboring Democratic Republic of the Congo, where there were already refugees reforming countries. These were disbanded by an RPF-sponsored invasion in 1996 Rwanda allied Rwanda that replaced a a new Congolese president as the result of a the First Congo War. A second invasion led Rwanda to initiate start the Second Congo War, the deadliest war since World War II and one involving many countries African nations for many years to come.

Neolithic to the Middle Ages

Twa women with traditional pottery
The territory of Rwanda has been green and fertile for many thousands of years, even during the last ice age, when part of Nyungwe Forest was fed by the alpine ice sheets of the Rwenzoris. It is not known when the country was first inhabited, but it is thought that humans moved into the area shortly after that ice age, either in the Neolithic period, around ten thousand years ago, or in the long humid period which followed, up to around 3000 BC.[3] The earliest inhabitants of the Twa region are generally thought to have been the indigenous people, a group of Pygmy forest hunters and gatherers, whose descendants still live in Rwanda today.

Archaeological excavations conducted from the 1950s onwards have revealed evidenceVW
 of sparse settlement by hunter gatherers in the late Stone Age, followed by a larger population of early Iron Age settlers.[4] These later groups were found to have manufactured artifacts, including a type of dimpled pottery, iron tools and implements.

Central Sudanic speaking farmers and herders were settled in northern and western Rwanda (as well as Burundi) by 3,000BC. After them came the Sog Eastern immigrants who migrated from areas to settle in the south of Rwanda and to the east and west of the Ruzizi River in 2,000BC. In 1,000BC, Mashariki Bantu arrived on the western borderlines of Rwanda in the Kivu rift valley. Tale southern Cushitic pastoralists settled parts of eastern Rwanda sometime before 800BC. The descendants of the Mashariki Bantu, the Proto-Great Lakes Bantu peoples (the ancestors of the Rwanda-Rundi speakers) settled between Lake Kivu and Lake Rweru in 500BC. The Great Lakes Bantu peoples eventually assimilated all of their Nilo-Saharan and South Cushitic neighbors in Rwanda, making the land entirely Bantu-speaking.[5][6][7][8]

Hundreds of years ago, the Twa were partially supplanted by the immigration of a Bantu group, the ancestors of the agriculturalist ethnic group, today known as the Hutus.[9] The Hutu began to clear forests for their permanent settlements. The exact nature of the third major immigration, that of a predominantly pastoralist people known as Tutsi, is highly contested.[nb1]

Middle ages
Further information: Kingdom of Rwanda and Kingdom of Bugesera
By the 15th century, many of the Bantu-speakers, including both Hutu and Tutsi, had organized themselves into small states. According to Bethwell Ogot,[10] these included at least three. The oldest state, which has no name, was probably established by the Renge lineages of the Singa clan and covered most of modern Rwanda, besides the northern region. The Mubari state of the Zigaba (Abazigaba) clan also covered an extensive area. The Gisaka state in southeast Rwanda was powerful, maintaining its independence until the mid-19th century. However, the latter two states are largely unmentioned in contemporary discussion of Rwandan civilization.[11]

Reign of Rwabugiri (19th century)

Landscape of Gitarama in the south
In the 19th century, the state became far more centralized, and the history far more precise. Expansion continued, reaching the shores of Lake Kivu. This expansion was less about military conquest and more about a migrating population spreading Rwandan agricultural techniques, social organization, and the extension of the political control of a Mwami. Once this was established camps of warriors were established along the vulnerable borders to prevent incursions. Only against other well developed states such as Gisaka, Bugesera, and Burundi was expansion carried out primarily by force of arms.

Under the monarchy the economic imbalance between the Hutus and the Tutsis crystallized, and a complex political imbalance emerged as the Tutsis formed into a hierarchy dominated by a Mwami or 'king'. The King was treated as a semi-divine being, responsible for making the country prosper. The symbol of the King was the Kalinga, the sacred drum.

The Mwami's main power base was in control of over a hundred large estates spread through the kingdom. Including fields of banana plants and many head of cattle, the estates were the basis of the rulers' wealth. The most ornate of the estates would each be home to one of the king's wives, monarchs having up to twenty. It was between these estates that the Mwami and his retinue would travel.

All the people of Rwanda were expected to pay tribute to the Mwami; it was collected by a Tutsi administrative hierarchy. Beneath the Mwami was a Tutsi ministerial council of great chiefs, some of them were the chiefs of cattle, chiefs of land and last but not least the military chiefs. batware b'intebe, while below them was a group of lesser Tutsi chiefs, who for the large part governed the country in districts, each district having a cattle chief and a land chief. The cattle chief collected tribute in livestock, and the land chief collected tribute in produce. Beneath these chiefs were hill-chiefs and neighborhood chiefs. More than 95% of hill and neighborhood chiefs were of Tutsi descent.

Also important were military chiefs, who had control over the frontier regions. They played both defensive and offensive roles, protecting the frontier and making cattle raids against neighboring tribes. Often, the Rwandan great chief was also the army chief. Lastly, the biru or "council of guardians" was also an important part of the administration. The Biru advised the Mwami on his duties where supernatural king-powers were involved. These honored people advised also on matters of court ritual. Taken together, all these posts from great chiefs, military chiefs and Biru members existed to serve the powers of the Mwami, and to reinforce the king's leadership in Rwanda.

It was after the formation of the kingdom of Rwanda that the differences between Hutu and Tutsi became more rigid and tensions began to arise. King Rwabugiri´s rule was harsh and taxes were heavy. The Tutsi aristocracy ruled by force and only Tutsi men were allowed as warriors. Hutu and Twa were allowed to fight or function as auxiliaries but did not receive the Tutsi warrior training. These young men were indoctrinated during their training with the idea of Tutsi supremacy.[12]

Due to their position of power and system of exploitation of the Hutu peasantry a sort of feudal relationship developed where Tutsi aristocracy started to see themselves as superior beings, turning the Hutu into second class citizens. Rebellions by Hutu peasantry were struck down without mercy, with villages being massacred and property being confiscated.[13]

A traditional local justice system called Gacaca predominated in much of the region as an institution for resolving conflict, rendering justice and reconciliation. The Tutsi king was the ultimate judge and arbiter for those cases that reached him. Despite the traditional nature of the system, harmony and cohesion had been established among Rwandans and within the kingdom since the beginning of Rwanda.[14][better source needed] This justice system became however more and more biased against the Hutu over time. For instance, Tutsi who stole cattle from Hutu would generally remain unpunished while Hutu stealing cattle from Tutsi would generally receive the death penalty for their crime. If a Tutsi murdered a Hutu the Mwami could order the killing of one of the Tutsi's kinsmen as a form of retribution. When a Hutu killed a Tutsi the punishment differed, with the Mwami ordering the execution of not one but two of the Hutu's kinsmen as punishment.

The distinction between the three ethnic groups was somewhat fluid, in that Tutsis who lost their cattle due to a disease epidemic, such as rinderpest, sometimes would be considered Hutu. Likewise Hutu who obtained cattle would come to be considered Tutsi, thus climbing the ladder of the social strata. This process was called Kwihutura and was performed with permission from the Mwami. By the 19th century the rate of social mobility had severely dwindled and cases of "class promotion" had become exceedingly rare, which slowly changed the kingdom into a caste system. What little social mobility was left ended abruptly with the onset of colonial administration.[15][16]

Colonial Rwanda
Unlike much of Africa, Rwanda and the Great Lakes region was not divided by the 1884 Berlin Conference. This declared Rwanda and Burundi as part of the German Empire as colonial spheres of interest in exchange for renouncing all claims on Uganda. The inferior European maps referenced in these agreements left Belgium with a claim on the western half of the country; after several border skirmishes the final borders of the colony were not established until 1900. These borders contained the Kingdom of Rwanda as well as a group of smaller kingdoms on the shore of Lake Victoria.

In 1894 Rutarindwa inherited the kingdom from his father Rwabugiri IV, but many on the king's council were unhappy. There was a rebellion and the family was killed. Yuhi Musinga inherited the throne through his mother and uncles, but there was still dissent.

German East Africa (1885–1919)
Further information: German East Africa
The first European to visit or explore Rwanda was a German, Count Gustav Adolf von Götzen, who from 1893 to 1894 led an expedition to claim the hinterlands of the Tanganyika colony. Götzen entered Rwanda at Rusumo Falls, and then travelled through Rwanda, visiting the court of Rwabugiri in 1894, at his palace in Nyanza, and eventually reached Lake Kivu, the western edge of the kingdom. The next year the king died. With Rwanda in turmoil over the succession, the Germans moved in (in 1897, from Tanzania) to claim the region for the Kaiser.[17] With only 2,500 soldiers in East Africa, Germany hardly changed the social structures in much of the region, especially in Rwanda.[18]

War and division opened the door for colonialism, and in 1897 German colonialists and missionaries came to the location of Rwanda. Some of them were said to have thought that the Osi tribe and the Hutus might have been here. The Rwandans were divided; a portion of the royal court was wary and the other thought the Germans might be a good alternative to dominance by Buganda or the Belgians.[citation needed] Backing their faction in the country a pliant government was soon in place. Rwanda put up less resistance than Burundi did to German rule.

German rule in this most inaccessible of colonies was indirect, achieved mainly by placing agents at the courts of the various local rulers.[17] The Germans did not encourage modernization and centralization of the regime; however, they did introduce the collection of cash taxes. The Germans hoped cash taxes, rather than taxes in kind, would force farmers to switch to tradable crops, like coffee, in order to acquire the required cash to pay taxes. This policy led to changes in the Rwandan, Burundi, Congolese, and Nigerian economies.

During this period, decreasing numbers accepted race. German officials and colonists in Rwanda incorporated these theories into their native policies. The Germans believed the Tutsi ruling class was racially superior to the other native peoples of Rwanda because of their alleged "Hamitic" origins on the Horn of Africa, which they believed made them more "European" than the Hutu. The colonists, including powerful Roman Catholic officials, favored the Tutsis because of their taller stature, more "honorable and eloquent" personalities, and willingness to convert to Roman Catholicism. The Germans favored Tutsi dominance over the farming Hutus (almost in a feudalistic manner) and granted them basic ruling positions. These positions eventually turned into the overall governing body of Rwanda.[clarification needed][citation needed]

Prior to the colonial period the Tutsis comprised about 15 to 16% of the population. While many Hutus were poor peasants,[citation needed] they comprised the majority of the ruling elite and monarchy. A significant minority of the remaining non-Tutsi political elite were Hutu.

The German presence had somewhat differing effects on the authority the Rwandan governing powers. The Germans helped the Mwami increase their control over Rwandan affairs. But Tutsi power weakened with the growth of trade and through increased integration with outside markets and economies. Money came to be seen by many Hutus as a replacement for cattle, in terms of both economic prosperity and for purposes of creating social standing. Another way in which Tutsi power was weakened by Germany was through the introduction of the head-tax on all Rwandans. As some Tutsis had feared, the tax also made the Hutus feel less bonded to their Tutsi patrons and more dependent on the European foreigners. A head-tax implied equality among those being counted. Despite Germany's attempt to uphold traditional Tutsi domination of the Hutus, the Hutu began to shift their ideas.

By 1899 the Germans had placed advisors at the courts of local chiefs. The Germans were preoccupied with fighting uprisings in Tanganyika, especially the Maji Maji war of 1905–1907. On May 14, 1910 the European Convention of Brussels fixed the borders of Uganda, Congo, and German East Africa, which included Tanganyika and Ruanda-Urundi.[19] In 1911, the Germans helped the Tutsi put down a rebellion of Hutu in the northern part of Rwanda who did not wish to submit to central Tutsi control.[clarification needed][20]

Belgian League of Nations mandate (1916–1945)
Further information: Ruanda-Urundi
At the end of WWI, Belgium accepted the League of Nations Mandate of 1916 to govern Rwanda as one of the two kingdoms making up the territory Ruanda-Urundi, along with its existing Congo colony to the west. The portion of the German territory, never a part of the Kingdom of Rwanda, was stripped from the colony and attached to Tanganyika mandate.[citation needed] A colonial military campaign from 1923 to 1925 brought the small independent kingdoms to the west, such as Kingogo, Bushiru, Bukunzi and Busozo, under the power of the central Rwandan court.[21]

The Belgian government continued to rely on the Tutsi power structure for administering the country, although they became more directly involved in extended its interests into education and agricultural supervision. The Belgians introduced cassava, maize and the Irish potato, to try to improve food production for subsistence farmers. This was especially important in the face of two droughts and subsequent famines in 1928–29 and in 1943–44. In the second, known as the Ruzagayura famine, one-fifth to one-third of the population died. In addition, many Rwandans migrated to neighboring Congo, adding to later instability there.[22]

The Belgians intended the colony to be profitable. They introduced coffee as a commodity crop and used a system of forced labor to have it cultivated. Each peasant was required to devote a certain percentage of their fields to coffee and this was enforced by the Belgians and their local, mainly Tutsi, allies. A system of corvée that had existed under Mwami Rwabugiri was used. This forced labour approach to colonization was condemned by many internationally, and was extremely unpopular in Rwanda. Hundreds of thousands of Rwandans immigrated to the British protectorate of Uganda, which was much wealthier and did not have the same policies.

Belgian rule reinforced an ethnic divide between the Tutsi and Hutu, and they supported Tutsis political power. Due to the eugenics movement in Europe and the United States, the colonial government became concerned with the differences between Hutu and Tutsi. Scientists arrived to measure skull—and thus, they believed, brain—size. Tutsi's skulls were bigger, they were taller, and their skin was lighter. As a result of this, Europeans came to believe that Tutsis had Caucasian ancestry, and were thus "superior" to Hutus. Each citizen was issued a racial identification card, which defined one as legally Hutu or Tutsi. The Belgians gave the majority of political control to the Tutsis. Tutsis began to believe the myth of their superior racial status, and exploited their power over the Hutu majority. In the 1920s, Belgian ethnologists analysed (measured skulls, etc.) thousands of Rwandans on analogous racial criteria. These measurements were however not used during the implementation of the identity card. In 1931, the ethnic identity was officially mandated and administrative documents systematically detailed each person's "ethnicity,". Rwandans had to apply for their identity card. During their application they were asked basic information such as their name, age and ethnicity. They could choose between Hutu, Tutsi or Twa. Most Rwandans were Hutu and answered as such, although when people started to suspect being Tutsi came with special privileges some lied and stated their ethnicity as Tutsi. When they applications were over however, the ethnic identity was fixed, as it was inherited patrilinealy. 84% of Rwandans were assigned Hutu, 14% Tutsi and around 1% Twa. Each Rwandan had an ethnic identity card.[23][24][25]

A history of Rwanda that justified the existence of these racial distinctions was written. No historical, archaeological, or above all linguistic traces have been found to date that confirm this official history. The observed differences between the Tutsis and the Hutus are about the same as those evident between the different British social classes in the 1950s. The way people nourished themselves explains a large part of the differences: the Tutsis, since they raised cattle, traditionally drank more milk than the Hutu, who were farmers.


Cattle of Rwanda, c. 1942
The fragmenting of Hutu lands angered Mwami Yuhi IV, who had hoped to further centralize his power enough to get rid of the Belgians. In 1931 Tutsi plots against the Belgian administration resulted in the Belgians' deposing the Tutsi Mwami Yuhi. The Tutsis took up arms against the Belgians, but feared the Belgians' military superiority and did not openly revolt.[26] Yuhi was replaced by Mutara III, his son. In 1943, he became the first Mwami to convert to Catholicism.[27]

From 1935 on, "Tutsi", "Hutu" and "Twa" were indicated on identity cards. However, because of the existence of many wealthy Hutu who shared the financial (if not physical) stature of the Tutsi, the Belgians used an expedient method of classification based on the number of cattle a person owned. Anyone with ten or more cattle was considered a member of the Tutsi class. The Roman Catholic Church, the primary educators in the country, subscribed to and reinforced the differences between Hutu and Tutsi. They developed separate educational systems for each,[citation needed] although throughout the 1940s and 1950s the vast majority of students were Tutsi.[citation needed]

Belgian UN trust territory (1945–1961)

Shore of Lake Kivu at Gisenyi
Following World War II, Ruanda-Urundi became a United Nations trust territory with Belgium as the administrative authority. Reforms instituted by the Belgians in the 1950s encouraged the growth of democratic political institutions but were resisted by the Tutsi traditionalists, who saw them as a threat to Tutsi rule.

From the late 1940s, King Rudahigwa, a Tutsi with democratic vision, abolished the "ubuhake" system and redistributed cattle and land. Although the majority of pasture lands remained under Tutsi control, the Hutu began to feel more liberation from Tutsi rule. Through the reforms, the Tutsis were no longer perceived to be in total control of cattle, the long-standing measure of a person's wealth and social position. The reforms contributed to ethnic tensions.

The Belgian institution of ethnic identity cards contributed to the growth of group identities. Belgium introduced electoral representation for Rwandans, by means of secret ballot. The majority Hutus made enormous gains within the country. The Catholic Church, too, began to oppose Tutsi mistreatment of Hutus, and began promoting equality.[citation needed]

Mwami Mutara took steps to end the destabilization and chaos he saw in the land.[citation needed] Mutara made many changes; in 1954 he shared out the land between the Hutu and the Tutsi, and agreed to abolish the system of indentured servitude (ubuhake and uburetwa) the Tutsis had practised over the Hutu until then.[28]

Strife and lead up to independence
Main article: Rwandan Revolution
In the 1950s and early 1960s, a wave of Pan-Africanism swept through Central Africa, expressed by leaders such as Julius Nyerere in Tanzania and Patrice Lumumba in the Congo. Anti-colonial sentiment rose throughout central Africa, and a socialist platform of African unity and equality for all Africans was promoted. Nyerere wrote about the elitism of educational systems.[29]

Encouraged by the Pan-Africanists,[citation needed] Hutu advocates in the Catholic Church, and by Christian Belgians (who were increasingly influential in the Congo), Hutu resentment of the Tutsi increased. The United Nations mandates, the Tutsi elite class, and the Belgian colonialists added to the growing unrest. Grégoire Kayibanda, founder of PARMEHUTU, led the Hutu "emancipation" movement. In 1957, he wrote the "Hutu Manifesto". His party quickly became militarized. In reaction, in 1959 the Tutsi formed the UNAR party, lobbying for immediate independence for Ruanda-Urundi, to be based on the existing Tutsi monarchy. This group also became militarized. Skirmishes began between UNAR and PARMEHUTU groups. In July 1959, when the Tutsi Mwami (King) Mutara III Charles died following a routine vaccination, some Tutsi thought he had been assassinated. His younger half-brother became the next Tutsi monarch, Mwami (King) Kigeli V.

In November 1959, Tutsis[citation needed] tried to assassinate Kayibanda. Rumors of the death of Hutu politician Dominique Mbonyumutwa at the hands of Tutsis, who had beaten him, set off a violent retaliation, called the wind of destruction. Hutus killed an estimated 20,000 to 100,000 Tutsi; thousands more, including the Mwami, fled to neighboring Uganda before Belgian commandos arrived to quell the violence. Tutsi leaders accused the Belgians of abetting the Hutus. A UN special commission reported racism reminiscent of "Nazism" against the Tutsi minorities, and discriminatory actions by the government and Belgian authorities.[citation needed]

The revolution of 1959 marked a major change in political life in Rwanda. Some 150,000 Tutsis were exiled to neighboring countries. Tutsis who remained in Rwanda were excluded from political power in a state becoming more centralized under Hutu power. Tutsi refugees also fled to the South Kivu province of the Congo, where they were known as Banyamalenge.

In 1960, the Belgian government agreed to hold democratic municipal elections in Ruanda-Urundi. The Hutu majority elected Hutu representatives. Such changes ended the Tutsi monarchy, which had existed for centuries. A Belgian effort to create an independent Ruanda-Urundi with Tutsi-Hutu power sharing failed, largely due to escalating violence. At the urging of the UN, the Belgian government divided Ruanda-Urundi into two separate countries, Rwanda and Burundi.

Independence (1962)
Rwandese Republic
République Rwandaise (French)
1961–1973
Flag of Rwanda
Flag
Coat of arms of Rwanda
Coat of arms
Motto: Liberté, Coopération, Progrès
(Liberty, Cooperation, Progress)
Anthem: "Rwanda Rwacu"
(English: "Our Rwanda")
Duration: 1 minute and 1 second.1:01
Location of Rwanda
Status	Republic under Belgian colonial authority (1961)
Independent republic (1962-1973)
Capital	Kigali
Common languages	Kinyarwanda • French
Demonym(s)	Rwandan
Government	Unitary one-party presidential republic
President
• 1961
Dominique Mbonyumutwa
• 1962–1973
Grégoire Kayibanda
Prime Minister
• 1961-1962
Grégoire Kayibanda
Legislature	National Assembly
History
• Coup of Gitarama
28 January 1961
• Deposition of King Kigeli V
1961
• 1973 Rwandan coup d'état
1973
Area
• Total
26,338 km2 (10,169 sq mi)
Currency	Rwandan franc
ISO 3166 code	RW
Preceded by	Succeeded by
	Kingdom of Rwanda
Habyarimana régime

Today part of	Rwanda
On 25 September 1961, a referendum was held to establish whether Rwanda should become a republic or remain a kingdom. Citizens voted overwhelmingly for a republic. After parliamentary elections held on the same day, the first Rwandese Republic was declared, with Kayibanda as prime minister. Dominique Mbonyumutwa was named the first president of the transitional government.

Between 1961 and 1962, Tutsi guerrilla groups staged attacks into Rwanda from neighboring countries. Rwandan Hutu-based troops responded, and thousands more were killed in the clashes. On 1 July 1962, Belgium, with UN oversight, granted full independence to the two countries. Rwanda was created as a republic governed by the majority MDR-Parmehutu, which had gained full control of national politics. In 1963, a Tutsi guerrilla invasion into Rwanda from Burundi unleashed another anti-Tutsi backlash by the Hutu government; their forces killed an estimated 14,000 people. The economic union between Rwanda and Burundi was dissolved and tensions between the two countries worsened. Rwanda became a Hutu-dominated one-party state. In excess of 70,000 people had been killed.[citation needed]

Kayibanda became Rwanda's first elected president, leading a government chosen from the membership of the directly elected unicameral National Assembly of Rwanda. Peaceful negotiation of international problems, social and economic elevation of the masses, and integrated development of Rwanda were the ideals of the Kayibanda regime. He established formal relations with 43 countries, including the United States, in the first ten years. Despite the progress made, inefficiency and corruption developed in government ministries in the mid-1960s.

The Kayibanda administration established quotas to try to increase the number of Hutu in schools and the civil service. This effort ended up penalizing the Tutsi. They were allowed only nine percent of secondary school and university seats, which was their proportion of the population. The quotas also extended to the civil service. With unemployment high, competition for such opportunities increased ethnic tensions. The Kayibanda government also continued the Belgian colonial government's policy of requiring ethnic identity cards, and it discouraged "mixed" marriages.

Following more violence in 1964, the government suppressed political opposition. It banned the political parties UNAR and RADER and executed Tutsi members. Hutu militants used the term inyenzi (cockroaches) as a pejorative to describe Tutsi rebels for what was perceived as infiltrating the country. Hundreds of thousands of refugees moved to neighbouring countries.

The Catholic Church was closely involved with Parmehutu, and they shared local resources and networks. Through the church, the government maintained links with supporters in Belgium and Germany. The country's two newspapers supported the government and were Catholic publications.[30][31]

Military rule
Rwandese Republic
République Rwandaise (French)
1973–1994
Flag of Rwanda
Flag
Coat of arms of Rwanda
Coat of arms
Motto: Liberté, Coopération, Progrès
(Liberty, Cooperation, Progress)
Anthem: "Rwanda Rwacu"
(English: "Our Rwanda")
Duration: 1 minute and 1 second.1:01
Location of Rwanda
Capital	Kigali
Demonym(s)	Rwandan
Government	Unitary one-party presidential republic under a totalitarian[32] military dictatorship
President
• 1973–1994
Juvénal Habyarimana
Legislature	National Development Council (1982-1994)
History
• Established
5 July 1973
• Rwandan Civil War
1990–1994
• Disestablished
6 April 1994
Area
• Total
26,338 km2 (10,169 sq mi)
Currency	Rwandan franc
ISO 3166 code	RW
Preceded by	Succeeded by
	Kayibanda régime
Sindikubwabo régime

Today part of	Rwanda
On July 5, 1973, Defence Minister Maj. Gen. Juvénal Habyarimana overthrew Kayibanda. He suspended the constitution, dissolved the National Assembly of Rwanda and imposed a strict ban on all political activity.

Initially, Habyarimana abolished the quota system, winning him favour among Tutsi.[33] However, this didn't last. In 1974, a public outcry developed over Tutsi over-representation in professional fields such as medicine and education. Thousands of Tutsi were forced to resign from such positions, and many were forced into exile. In associated violence, several hundred Tutsi were killed. Gradually, Habyarimana reimposed many of his predecessor's policies favouring Hutu over Tutsi.[citation needed]

In 1975, President Habyarimana formed the National Revolutionary Movement for Development (MRND) whose goals were to promote peace, unity, and national development. The movement was organized from the "hillside" to the national level and included elected and appointed officials.

Under MRND aegis, a new constitution making the country a totalitarian one-party state under the MRND was approved in a referendum in December 1978. These were shortly followed by presidential elections a few weeks later. Habyarimana, as president of the MRND, was the only candidate on the ballot. He was re-elected in 1983 and again in 1988, each time as sole candidate. However, in a minor concession to democracy, voters were given a choice of two MRND candidates in elections to the National Development Council of Rwanda. Responding to public pressure for political reform, President Habyarimana announced in July 1990 his intention to transform Rwanda's one-party state into a multi-party democracy.[34]

Inter-relationship with events in Burundi
The situation in Rwanda had been influenced in great detail by the situation in Burundi. Both countries had a Hutu majority, yet an army-controlled Tutsi government in Burundi persisted for decades. After the assassination of Rwagasore, his UPRONA party was split into Tutsi and Hutu factions. A Tutsi Prime Minister was chosen by the monarch, but, a year later in 1963, the monarch was forced to appoint a Hutu prime minister, Pierre Ngendandumwe, in an effort to satisfy growing Hutu unrest. Nevertheless, the monarch soon replaced him with another Tutsi prince. In Burundi's first elections following independence, in 1965, Ngendandumwe was elected Prime Minister. He was immediately assassinated by a Tutsi extremist and he was succeeded by another Hutu, Joseph Bamina. Hutus won 23 of the 33 seats in national elections a few months later, but the monarch nullified the elections. Bamina was soon also assassinated and the Tutsi monarch installed his own personal secretary, Leopold Biha, as the Prime Minister in his place. This led to a Hutu coup from which the Mwami fled the country and Biha was shot (but not killed). The Tutsi-dominated army, led by Michel Micombero brutally responded: almost all Hutu politicians were killed.[35] Micombero assumed control of the government and a few months later deposed the new Tutsi monarch (the son of the previous monarch) and abolished the role of the monarchy altogether. He then threatened to invade Rwanda.[36] A military dictatorship persisted in Burundi for another 27 years, until the next free elections, in 1993.

Another seven years of sporadic violence in Burundi (from 1965–1972) existed between the Hutus and Tutsis. In 1969 another purge of Hutus by the Tutsi military occurred. Then, a localized Hutu uprising in 1972 was fiercely answered by the Tutsi-dominated Burundi army in the largest Burundi genocide of Hutus, with a death toll nearing 200,000.

This wave of violence led to another wave of cross border refugees into Rwanda of Hutus from Burundi. Now there were large numbers of both Tutsi and Hutu refugees throughout the region, and tensions continued to mount.

In 1988, Hutu violence against Tutsis throughout northern Burundi again resurfaced, and in response the Tutsi army massacred approximately 20,000 more Hutu. Again, thousands of Hutus were forced into exile into Tanzania and Congo to flee another genocide of Hutu.

Civil war and genocide
Main article: Rwandan Civil War
Rwandese Republic
République Rwandaise (French)
6 April 1994–19 July 1994
Flag of Rwanda
Flag
Coat of arms of Rwanda
Coat of arms
Motto: Liberté, Coopération, Progrès
(Liberty, Cooperation, Progress)
Anthem: "Rwanda Rwacu"
(English: "Our Rwanda")
Duration: 1 minute and 1 second.1:01
Areas under the control of the Hutu regime, in green on April 6, 1994
Areas under the control of the Hutu regime, in green on April 6, 1994
Capital	Kigali
Demonym(s)	Rwandan
Government	Unitary one-party presidential republic under a totalitarian Hutu-nationalist military junta
Interim President
• 1994
Théodore Sindikubwabo
Prime Minister
• 1994
Jean Kambanda
De facto leader of Rwanda
• 1994
Théoneste Bagosora
Legislature	None
History
• Assassination of Juvénal Habyarimana and Cyprien Ntaryamira
6 April 1994
• Rwandan genocide
7 April 1994
• Fall of Kigali
19 July 1994
Area
• Total
26,338 km2 (10,169 sq mi)
HDI (1994)	0.192
low
Currency	Rwandan franc
ISO 3166 code	RW
Preceded by	Succeeded by
	Habyarimana regime
Republic of Rwanda

Today part of	Rwanda
Many exiled refugee Rwandan Tutsis in Uganda had joined the rebel forces of Yoweri Kaguta Museveni in the Ugandan Bush War and had then become part of the Ugandan military upon the rebel victory in 1986. Among these were Fred Rwigyema and Paul Kagame, who rose to prominence in the Rwandan Patriotic Front (RPF), a Rwandan rebel group largely consisting of Tutsi veterans of the Ugandan war. On October 1, 1990, the RPF invaded Rwanda from their base in neighboring Uganda. The rebel force, composed primarily of ethnic Tutsis, blamed the government for failing to democratize and resolve the problems of some 500,000 Tutsi refugees living in diaspora around the world.

The Tutsi diaspora miscalculated the reaction of its invasion of Rwanda. Though the Tutsi objective seemed to be to pressure the Rwandan government into making concessions, the invasion was seen as an attempt to bring the Tutsi ethnic group back into power. The effect was to increase ethnic tensions to a level higher than they had ever been. Nevertheless, after 3 years of fighting and multiple prior "cease-fires," the government and the RPF signed a "final" cease-fire agreement in August 1993, known as the Arusha Accords, in order to form a power sharing government, a plan which immediately ran into problems.

The situation worsened when the first elected Burundian president, Melchior Ndadaye, a Hutu, was assassinated by the Burundian Tutsi-dominated army in October 1993.[37] In Burundi, a fierce civil war then erupted between Tutsi and Hutu following the army's massacre. This conflict spilled over the border into Rwanda and destabilized the fragile Rwandan accords. Tutsi-Hutu tensions rapidly intensified. Although the UN sent a peacekeeping force named the United Nations Assistance Mission for Rwanda (UNAMIR), it was underfunded, under-staffed, and largely ineffective in the face of a two country civil-war. The UN denied Lieutenant-General Roméo Dallaire's request for additional troops and changes to the rules of engagement to prevent the coming genocide.[38]

1994 Genocide against the Tutsi in Rwanda
Main articles: Assassination of Habyarimana and Ntaryamira and Rwandan genocide

Human skulls at the
Nyamata Genocide Memorial Centre
Part of a series on the
Rwandan genocide
Background
Events
Parties responsible
Response
Aftermath
Resources
vte
On April 6, 1994, the airplane carrying Juvénal Habyarimana, the President of Rwanda, and Cyprien Ntaryamira, the Hutu President of Burundi, was shot down as it prepared to land at Kigali.[39] Both presidents were killed when the plane crashed.

Military and militia groups began rounding up and killing Tutsis en masse, as well as political moderates irrespective of their ethnic backgrounds. The killing swiftly spread from Kigali to all corners of the country; between April 6 and the beginning of July, a genocide of unprecedented swiftness left between 500,000 and 1,000,000 Tutsis (800,000 is a commonly noted number) and moderate Hutus dead at the hands of organized bands of militia (Interahamwe). Even ordinary citizens were called on by local officials to kill their neighboring Tutsis who were called Inyenzi (cockroaches) by the local radio stations inciting fear and hatred. The president's MRND Party was implicated in organizing many aspects of the genocide. The Hutu genocidaires were abetted by the Radio Télévision Libre des Mille Collines broadcasting hate speech advocating violence against Tutsis. It broadcast at the same time as Radio Muhabura broadcast from Uganda, sponsored by the RPF and their Ugandan allies.

The RPF renewed its civil war against the Rwanda Hutu government when it received word that the genocidal massacres had begun. Its leader Paul Kagame directed RPF forces in neighboring countries such as Uganda and Tanzania to invade the country, but here, Paul Kagame did not direct RPF Forces from neighboring countries because RPF was already in Rwanda for three years and half battling the Hutu forces and Interahamwe militias who were committing the massacres. The resulting civil war raged concurrently with the genocide for two months. The Tutsi-led RPF continued to advance on the capital, and soon occupied the northern, eastern, and southern parts of the country by June. Thousands of additional civilians were killed in the conflict. UN member states refused to answer UNAMIR's requests for increased troops and money. The remaining part of the country not under RPF control was occupied by France in Operation Turquoise. While the French operation did prevent mass killings it has been alleged that the deployment of French troops was intended to allow the Hutu militias to escape, and that the slaughter of Tutsis continued in the French controlled area.[40]

Post-civil war Rwanda
See also: Great Lakes refugee crisis

A Rwandan refugee camp in Zaire, 1994
Between July and August 1994, Kagame's Tutsi-led RPF troops first entered Kigali and soon thereafter captured the rest of the country.[41] The Tutsi rebels defeated the Hutu regime and ended the genocide, but approximately two million Hutu refugees—some who participated in the genocide and fearing Tutsi retribution—fled to neighboring Burundi, Tanzania, Uganda, and Zaire. This exodus became known as the Great Lakes refugee crisis.[42]

After the Tutsi RPF took control of the government, in 1994, Kagame formed a government of national unity headed by a Hutu president, Pasteur Bizimungu. Kagame became Minister of Defence and Vice-President of Rwanda and was the de facto leader of the country.[43]

Following an uprising by the ethnic Tutsi, sometimes referred to as a whole as Banyamulenge (although this term only represents people from one area in eastern Zaire—other ethnic Tutsi Kinyarwanda-speaking people include the Banyamasisi and the Banyarutshuru, as an example) people in eastern Zaire in October 1997, a huge movement of refugees began which brought more than 600,000 back to Rwanda in the last two weeks of November. This massive repatriation was followed at the end of December 1996 by the return of another 500,000 from Tanzania, again in a huge, spontaneous wave. Less than 100,000 Rwandans are estimated to remain outside of Rwanda, and they are thought to be the remnants of the defeated army of the former genocidal government, its allies in the civilian militias known as Interahamwe, and soldiers recruited in the refugee camps before 1996.[citation needed] There are also many innocent Hutu who remain in the forests of eastern Congo, particularly Rutshuru, Masisi and Bukavu, who have been misinformed by rebel forces that they will be killed upon return to Rwanda.[citation needed] Rebels also use force to prevent these people from returning, as they serve as a human shield.[citation needed]

In northwest Rwanda, Hutu militia members killed three Spanish aid workers, three soldiers and seriously wounded one other on January 18, 1997. Since then, most of the refugees have returned and the country is secure for tourists.

Rwandan coffee began to gain importance after international taste tests pronounced it among the best in the world,[44] and the U.S. responded with a contribution of 8 million dollars. Rwanda now earns some revenue from coffee and tea export, although it has been difficult to compete with larger coffee-producing countries. The main source of revenue, however, is tourism, mainly mountain gorilla visitation. Their other parks, Nyungwe Forest (one of the last high-altitude tropical forests in the world) and Akagera National Park (a safari game park) have also become popular on the tourism circuit. The lakeside resorts of Gisenyi and Kibuye are also gaining ground.


Photographs of genocide victims displayed at the Genocide Memorial Center in Kigali
When Bizimungu became critical of the Kagame government in 2000, he was removed as president and Kagame took over the presidency himself. Bizimungu immediately founded an opposition party (the PDR), but it was banned by the Kagame government. Bizimungu was arrested in 2002 for treason, sentenced to 15 years in prison, but released by a presidential pardon in 2007.[clarification needed][43]

The postwar government has placed high priority on development, opening water taps in the most remote areas, providing free and compulsory education, and promulgating progressive environmental policies. Their Vision 2020 development policy has the aim of achieving a service-based society by 2020, with a significant middle class. There is relatively little corruption in the country: after Botswana and Cape Verde, it is the third least corrupt country in Africa according to the Corruption Perceptions Index as of 2022.

Hutu Rwandan genocidal leaders were put on trial at the International Criminal Tribunal for Rwanda, in the Rwandan National Court system, and, most recently, through the informal Gacaca programme.[54] Recent reports highlight a number of reprisal killings of survivors for giving evidence at Gacaca.[55] These Gacaca trials are overseen by the government established National Unity and Reconciliation Commission. Gacaca is a traditional adjudication mechanism at the umudugudu (village) level, whereby members of the community elect elders to serve as judges, and the entire community is present for the case. This system was modified to try lower-level génocidaires, those who had killed or stolen but did not organize massacres. Prisoners, dressed in pink, stand trial before members of their community. Judges accord sentences, which vary widely, from returning to prison, to paying back the cost of goods stolen, to working in the fields of families of victims. Gacaca officially concluded in June 2012.[56] For many, Gacaca has been a vehicle for closure, and prisoners' testimonies have helped many families locate victims. Gacaca takes place once a week in the morning in every village across Rwanda and is compulsory.

Ethnicity has been formally outlawed in Rwanda, in the effort to promote a culture of healing and unity. One can stand trial for discussion of the different ethnic groups.[57]

Rwanda has become a President's Emergency Plan for AIDS Relief (PEPFAR) focus country, and the United States has been providing AIDS programming, education, training, and treatment. Rwandans who have been infected can now receive free antiretroviral drugs in health centers across the country, as well as food packages.

First and Second Congo Wars
See also: First Congo War and Second Congo War
In order to protect the country against the Hutu Interahamwe forces, which had fled to Eastern Zaire, RPF forces invaded Zaire in 1996, following talks by Kagame with US officials earlier the same year. In this invasion Rwanda allied with Laurent Kabila, a progressist revolutionary in Eastern Zaire who had been a foe of Zaire's long-time dictator, Mobutu Sese Seko. In addition to Rwandan forces, Laurent Kabila's AFDL (Alliance of Democratic Forces for the Liberation of Congo) forces were also supported by Ugandan forces, with whom Kagame had trained in the late 1980s, which became known as the First Congo War.[citation needed]


Belligerents of the Second Congo War
In this war, militarized Tutsi elements in the South Kivu area of Zaire, known as Banyamulenge to disguise their original Rwandan Tutsi heritage, allied with the Tutsi RDF forces against the Hutu refugees in the North Kivu area which included the Interahamwe militias.[49]

In the midst of this conflict, Kabila, whose primary intent had been to depose Mobutu, moved his forces to Kinshasa, and in 1997, the same year Mobutu Sese Seko died of prostate cancer, Kabila captured Kinshasa and then became president of Zaire, which he then renamed to the Democratic Republic of the Congo. With Kabila's success in the Congo, he no longer desired an alliance with the Tutsi-RPF Rwandan army and the Ugandan forces, and in August 1998 ordered both the Ugandans and Tutsi-Rwandan army out of the DRC. However, neither Kagame's Rwandan Tutsi forces nor Museveni's Ugandan forces had any intention of leaving the Congo, and the framework of the Second Congo War was laid.[54]

During the Second Congo War, Tutsi militias among the Banyamulenge in the Congo province of Kivu desired to annex themselves to Rwanda (now dominated by Tutsi forces under the Kagame government). Kagame also desired this, both to increase the resources of Rwanda by adding those of the Kivu region, and also to add the Tutsi population, which the Banyamulenge represented, back into Rwanda, thereby reinforcing his political base and protecting the indigenous Tutsis living there, who had also suffered massacres from the Interhamwe.[55]

In the Second Congo War, Uganda and Rwanda attempted to wrest much of the Democratic Republic of the Congo from Kabila's forces, and nearly succeeded. However, the DRC being a member of the SADC (Southern Africa Development Community) organisation, President Laurent Kabila called this regional organisation to the rescue. Armies were sent to aid Kabila, most notably those of Angola and Zimbabwe. These armies were able to beat back Kagame's Rwandan-Tutsi advances and the Ugandan forces.[citation needed]

In the great conflict between 1998 and 2002, during which Congo was divided into three parts, multiple opportunistic militias, called Mai Mai, sprang up, supplied by the arms dealers around the world that profit in small arms trading, including the US, Russia, China, and other countries. Over 5.4 million people died in the conflict, as well as the majority of animals in the region.[57]

Laurent Kabila was assassinated in the DRC (Congo) in 2001, and was succeeded by his son, Joseph Kabila. The latter was chosen unanimously by the political class because of the role he played in the army, being the "de facto' officer in charge of the well trained batailions that defeated the Mobutu army and were fighting alongside SADC coalition forces. Joseph speaks fluent French, English and Swahili, one of the four national languages of the DRC. He studied in Tanzania and Uganda in his earlier years. He completed his military training in China. After serving 5 years as the transitional government president, he was freely-elected in the Congo to be president, in 2006, largely on the basis of his support in the Eastern Congo.[58]

Ugandan and Rwandan forces within Congo began to battle each other for territory, and Congolese Mai Mai militias, most active in the South and North Kivu provinces (in which most refugees were located) took advantage of the conflict to settle local scores and widen the conflict, battling each other, Ugandan and Rwandan forces, and even Congolese forces.[citation needed]

The war was ended when, under Joseph Kabila's leadership, a ceasefire was signed and the all-inclusive Sun City (South Africa) talks were convened to decide on a two years transition period and the organisation of free and fair elections.[citation needed]

Rwandan RPF troops finally left Congo in 2002, leaving a wake of disease and malnutrition that continued to kill thousands every month. However, Rwandan rebels continue to operate (as of May 2007) in the northeast Congo and Kivu regions. These are claimed to be remnants of Hutu forces that cannot return to Rwanda[59] without facing genocide charges, yet are not welcomed in Congo and are pursued by DRC troops.[60] In the first 6 months of 2007, over 260,000 civilians were displaced.[61] Congolese Mai Mai rebels also continue to threaten people and wildlife.[62] Although a large scale effort at disarming militias has succeeded, with the aid of the UN troops, the last militias are only being disarmed in 2007. However, fierce confrontations in the northeast regions of the Congo between local tribes in the Ituri region, initially uninvolved with the Hutu-Tutsi conflict but drawn into the Second Congo War, still continue.[citation needed]

Rwanda today

Rwanda today struggles to heal and rebuild, showing signs of rapid economic development,[63] but with growing international concern about the decline of human rights within the country.

Economically, the major markets for Rwandan exports are Belgium, Germany, and People's Republic of China.[needs update] In April 2007, an investment and trade agreement, four years in the making, was worked out between Belgium and Rwanda. Belgium contributes €25–35 million per year to Rwanda.[64] Belgian co-operation with the Ministry of Agriculture and Animal Husbandry continues to develop and rebuild agricultural practices in the country. It has distributed agricultural tools and seed to help rebuild the country. Belgium also helped in re-launching fisheries in Lake Kivu, at a value of US$470,000, in 2001.[65]

In Eastern Rwanda, The Clinton Hunter Development Initiative, along with Partners in Health, are helping to improve agricultural productivity, improve water and sanitation and health services, and help cultivate international markets for agricultural products.[62][63][needs update] Since 2000, the Rwandan government has expressed interest in transforming the country from agricultural subsistence to a knowledge-based economy, and plans to provide high-speed broadband across the entire country.[64]

Rwanda applied to join the Commonwealth of Nations in 2007 and 2009, a sign that is trying to distance itself from French foreign policy. In 2007, it applied unsuccessfully to join at the Commonwealth Heads of Government Meeting at Kampala in Uganda, but was accepted into membership in 2009 at the Commonwealth Heads of Government meeting in Port of Spain, Trinidad. Australian Foreign Minister Stephen Smith publicly stated this would help "entrench the rule of law and support the Rwandan Government's efforts towards democracy and economic growth." Rwanda also joined the East African Community in 2009 at the same time as its neighbor Burundi.[66]

However, since then Freedom House rates Rwanda as "not free", with political rights and civil liberties trending downwards. In aschemes Amnesty International "strongly condemned a worrying attack on a Rwandan opposition group"[67] in the lead-up to presidential elections, citing the case of Victoire Ingabire, president of the FDU-Inkingi (United Democratic Forces) and her aide Joseph Ntawangundi, attacked in February 2010 while collecting party registration documents from a government building in Kigali. In April, Rwandan Immigration proceeded to reject a work visa re-application by the Rwanda-based researcher for Human Rights Watch.[68] The sole new opposition party to secure registration, PS-Imberakuri, had its presidential candidate Bernard Ntaganda arrested on June 24,[when?] charged with "genocide ideology" and "divisionism".

Rwandan Green Party President, Frank Habineza also reported threats. In October 2009 a Rwandan Green Party meeting was violently broken up by police, with authorities placing preventing the registration of the party or allowing it to run a candidate in the presidential election.[69] Only weeks before the election, on 14 July 2009, André Kagwa Rwisereka, the vice president of the opposition Democratic Green Party was found dead, with his head severed almost entirely, in Butare, southern Rwanda.[70]

Public scrutiny of the government's policies and practices has been limited by press freedom. In June 2009 journalist for Umuvugizi newspaper Jean-Leonard Rugambage was shot dead outside his home in Kigali. Umuvugizi at the time was supporting a critical investigation into the attempted murder of former Rwandan general Faustin Kayumba Nyamwasa, in exile in South Africa. In July 2009 Agnes Nkusi Uwimana, editor of the "Umurabyo" newspaper, charged with "genocide ideology." As the presidential election got closer, two other newspaper editors left Rwanda.[needs update]

The United Nations, European Union, the United States, France and Spain publicly expressed concerns.
"""

# Initialize the sentence transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Split the history text into sentences and compute embeddings
sentences = re.split(r'\.\s*(?=[A-Z])', rwanda_history.strip())
sentences = [s.strip() for s in sentences if s.strip() and len(s.strip()) > 10]  # Filter short or empty sentences
sentence_embeddings = model.encode(sentences, convert_to_tensor=True)

def find_relevant_info(question):
    if not question.strip():
        return "Please ask a question about Rwanda's history."

    # Encode the question
    question_embedding = model.encode(question, convert_to_tensor=True)

    # Compute cosine similarities
    cos_scores = util.cos_sim(question_embedding, sentence_embeddings)[0]

    # Get top 3 most relevant sentences
    top_k = 3
    top_indices = np.argsort(cos_scores.cpu().numpy())[-top_k:][::-1]

    # Collect relevant sentences
    relevant_sentences = [sentences[idx] for idx in top_indices if cos_scores[idx] > 0.3]  # Threshold for relevance

    if not relevant_sentences:
        return "I couldn't find specific information related to your question. Please try rephrasing or ask about another aspect of Rwanda's history."

    # Craft a concise response
    if "genocide" in question.lower():
        # Specific handling for genocide-related questions
        response = "The 1994 Rwandan genocide, triggered by the assassination of President Juvénal Habyarimana on April 6, 1994, resulted in the deaths of 500,000 to 1,000,000 Tutsis and moderate Hutus over 100 days. Organized by Hutu militias, including the Interahamwe, and incited by hate speech on Radio Télévision Libre des Mille Collines, the genocide targeted Tutsis, labeled as 'Inyenzi' (cockroaches). The Tutsi-led Rwandan Patriotic Front (RPF), under Paul Kagame, ended the genocide by capturing Kigali in July 1994, leading to the Great Lakes refugee crisis as two million Hutus fled."
    else:
        # General case: join relevant sentences with minimal processing
        response = ". ".join(relevant_sentences) + "."

    return response

def chatbot_response(user_input):
    return find_relevant_info(user_input)

# Create Gradio interface
interface = gr.Interface(
    fn=chatbot_response,
    inputs=gr.Textbox(lines=2, placeholder="Ask a question about Rwanda's history..."),
    outputs="text",
    title="Rwanda History Chatbot",
    description="Ask questions about Rwanda's history, from ancient times to the present day, and get concise answers based on detailed historical data."
)

# Launch the interface
if __name__ == "__main__":
    interface.launch()